In [78]:
import os, cv2
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

    0 = spontanic
    1 = sync
    2 = alone

In [4]:
def encode(path, num):
    df = pd.read_csv(path).iloc[8:,:]
      
    for col in df.columns:
        col1 = col.replace(" ", "_")
        df = df.rename({col: col1}, axis=1)
        
    df = df.rename({'_#_hands': 'hands_num'}, axis=1)  
     
    if(num != 2):
        df = df[df.hands_num == 2]
        df = df.sort_values(by=['_Hand_Type'])
        left, right =split(df)
    else:
        left = df[df.hands_num == 1].reset_index(drop=True)
        right = HandRight.iloc[8:,:]
        if len(left)>len(right):
            right = right.append(right.iloc[8:(len(left)-len(right)+8),:]).reset_index(drop=True)
        else:
            right = right.iloc[:len(left),:]
        
    left = rename(left, 'L')
    right = rename(right, 'R')
    df = pd.concat([left, right], axis=1)
    df = df.drop(['LTime', 'RTime', 'L_Frame_ID', 'R_Frame_ID', 'Lhands_num', 'Rhands_num'], axis=1)
    
    df = df[df.index % 5 == 0].reset_index(drop=True)
    df = combine(df,3)
    
    df['label']=num
    
    return df

In [5]:
def split(df) :
    half = int(len(df)/2)
    hd = df.head(half)
    tl = df.tail(len(df)-half)
    hd = hd.reset_index(drop=True)
    tl = tl.reset_index(drop=True)
    return hd, tl


In [6]:
def rename(df, char):
    for col in df.columns:
        col1 = char+col
        df = df.rename({col: col1}, axis=1)
    return df

In [7]:
HandRight = pd.read_csv(r'HandRight.csv').iloc[8:,:]
for col in HandRight.columns:
    col1 = col.replace(" ", "_")
    HandRight = HandRight.rename({col: col1}, axis=1)
HandRight = HandRight.rename({'_#_hands': 'hands_num'}, axis=1) 
HandRight = HandRight[HandRight._Hand_Type == 'right']
HandRight = HandRight.dropna().reset_index(drop=True)

In [8]:
def combine(df,numOfRows):
    
    for i in range(1, numOfRows+1):
        line = ["{}{}".format(ind,'_'+str(i)) for ind in df.columns]
        if i==1:
            lineTotal = line
            lineArr = [line]
        else:
            lineTotal = lineTotal + line
            lineArr.append(line)

    headlines = pd.DataFrame(columns = lineTotal)

    
    for i in range(0, len(df)-len(df)%numOfRows, numOfRows):
    
        for j in range(0, numOfRows):
            listj = df.loc[i+j]
            listj.index = lineArr[j]
            if j==0:
                arr = [listj]
            else:
                arr.append(listj)
    
        jumpRows = pd.concat(arr)
        headlines = headlines.append(jumpRows, ignore_index = True)
        
    return headlines

In [9]:
#evyatar
ev_spo = encode(r'Evyatar636771052727603804Spontan.csv',0)
ev_sync = encode(r'Evyatar636771053639929594Sync.csv',1)
ev_al = encode(r'Evyatar636771054555711409Alone.csv',2)

In [10]:
#nofar
nofar_spo = encode(r'Nofar636759795182793299Spontan.csv',0)
nofar_sync = encode(r'Nofar636759797397919664Sync.csv',1)
nofar_al = encode(r'Nofar636759796290435160Alone.csv',2)

In [11]:
#oriya
oriya_spo = encode(r'Oriya636759804404113837Spontan.csv',0)
oriya_sync = encode(r'Oriya636759806131350399Sync.csv',1)
oriya_al = encode(r'Oriya636759805268396661Alone.csv',2)


In [12]:
#orya
orya_spo = encode(r'OryaB636771083605535985Spontan.csv',0)
orya_sync = encode(r'OryaB636771084494502229Sync.csv',1)
orya_al = encode(r'OryaB636771082736601528Alone.csv',2)


In [13]:
#revital
rev_spo = encode(r'Revital636770957038873193Spontan.csv',0)
rev_sync = encode(r'Revital636770957894370062Sync.csv',1)
rev_al = encode(r'Revital636770955999435879Alone.csv',2)


In [14]:
#sachar
sachar_spo = encode(r'Sachar636771045591258071Spontan.csv',0)
sachar_sync = encode(r'Sachar636771046527112972Sync.csv',1)
sachar_al = encode(r'Sachar636771047408504303Alone.csv',2)


In [15]:
#shelly
shelly_spo = encode(r'Shelly636759787508979097Spontan.csv',0)
shelly_sync = encode(r'Shelly636759789526313406Sync.csv',1)
shelly_al = encode(r'Shelly636759788383205274Alone.csv',2)


In [16]:
#yael
yael_spo = encode(r'Yael636771059544754823Spontan.csv',0)
yael_sync = encode(r'Yael636771060699529085Sync.csv',1)
yael_al = encode(r'Yael636771061659067649Alone.csv',2)


In [17]:
#yoel
yoel_spo = encode(r'Yoel636760921663075813Spontan.csv',0)
yoel_sync = encode(r'Yoel636760923490742912Sync.csv',1)
yoel_al = encode(r'Yoel636760922573364655Alone.csv',2)

In [18]:
dataFrames = [ev_spo, nofar_spo, oriya_spo, orya_spo,rev_spo, sachar_spo, shelly_spo, yael_spo, yoel_spo,
         ev_sync, nofar_sync, oriya_sync, orya_sync,rev_sync, sachar_sync, shelly_sync, yael_sync, yoel_sync,
         ev_al, nofar_al, oriya_al, orya_al,rev_al, sachar_al, shelly_al, yael_al, yoel_al ]

df = pd.concat(dataFrames)

In [46]:
# Hand_Type from string to int: left = 0, right = 1
for col in df.columns:    
    if("Hand_Type" in col):
        if ("L" in col):
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(np.int64)
        else:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(1).astype(np.int64)

In [52]:
df.head()

,L_Hand_Type_1,L_Position_X_1,L_Position_Y_1,L_Position_Z_1,L_Velocity_X_1,L_Velocity_Y_1,L_Velocity_Z_1,L_Pitch_1,L_Roll_1,L_Yaw_1,...,R_Wrist_Pos_X_3,R_Wrist_Pos_Y_3,R_Wrist_Pos_Z_3,R_Elbow_pos_X_3,R_Elbow_Pos_Y_3,R_Elbow_Pos_Z_3,R_Grab_Strenth_3,R_Grab_Angle_3,R_Pinch_Strength_3,label
0,0,-102.09200,198.4801,36.821470,-143.442100,162.22250,138.81830,1.044945,0.220119,1.257915,...,205.1825,175.7701,14.586860,355.5135,38.01317,151.41010,0.0,0.000000,0.000000,0
1,0,-87.01425,181.4484,2.391368,0.267130,-39.35226,-14.64911,1.590843,0.713496,1.588960,...,149.3343,195.8305,18.134420,303.8842,50.75324,142.53030,0.0,0.600372,0.000000,0
2,0,-88.35677,117.5149,-1.507066,4.879696,-106.04380,30.13123,1.388725,0.532217,1.460566,...,165.8512,289.5272,23.289340,309.1972,101.78370,91.19925,0.0,0.454846,0.000000,0
3,0,-91.82849,187.9625,3.546683,-16.356640,-57.70452,-37.83114,1.616455,0.778057,1.621224,...,192.6499,207.2711,-2.155823,310.6127,30.43638,121.75680,0.0,0.908594,0.191551,0
4,0,-88.36761,185.6821,1.074904,9.990860,24.07274,-18.26352,1.561024,0.834355,1.560259,...,157.4693,267.7648,18.064220,309.6221,89.76556,92.71830,0.0,0.215488,0.000000,0


In [64]:
df = df.dropna()

In [65]:
y = df['label']
X = df.drop(['label'], axis=1)

In [66]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.values)

In [67]:
pca = PCA(n_components=0.9)

In [68]:
X_reduced = pca.fit_transform(X)

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42, shuffle = True)

In [73]:
Kfolds = StratifiedKFold(n_splits=5,shuffle=True, random_state=7)

In [74]:
model_params = {
    'random_forest':{
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [100, 150],
            'max_depth': [10,20,50,80,100]
        }
    },
    
    'logistic_regression':{
        'model': LogisticRegression(multi_class= 'auto'),
        'params': {
            'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
            'C': [0.1, 0.5, 1.0]
        }
    },
    
    'xgb':{
        'model': XGBClassifier(eval_metric='mlogloss'),
        'params':{
            'n_estimators': [100, 150],
            'learning_rate': [0.0001, 0.01, 0.2, 0.3]            
        }
    },
    
    'knn':{
        'model': KNeighborsClassifier(),
        'params': {
            'n_neighbors': [5,10, 15],
            'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
        }
    }
}

In [80]:
scores = []

for model_name, mp in model_params.items():
    clf = GridSearchCV(mp['model'], mp['params'], cv = Kfolds, return_train_score= False)
    clf.fit(X_train, y_train)
    scores.append({
        'model': model_name,
        'best score': clf.best_score_,
        'best params': clf.best_params_
    })

C:\Users\PC\anaconda3\lib\site-packages\sklearn\utils\optimize.py:211: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn("newton-cg failed to converge. Increase the "
C:\Users\PC\anaconda3\lib\site-packages\sklearn\utils\optimize.py:211: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn("newton-cg failed to converge. Increase the "
C:\Users\PC\anaconda3\lib\site-packages\sklearn\utils\optimize.py:211: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn("newton-cg failed to converge. Increase the "
C:\Users\PC\anaconda3\lib\site-packages\sklearn\utils\optimize.py:211: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn("newton-cg failed to converge. Increase the "
C:\Users\PC\anaconda3\lib\site-packages\sklearn\utils\optimize.py:211: ConvergenceWarning: newton-cg failed to converge. Increas

C:\Users\PC\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\PC\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

C:\Users\PC\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\PC\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\PC\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\PC\anaconda3\lib\site-packages\sklearn\utils\optimize.py:211: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn("newton-cg failed to converge. Increase the "
C:\Users\PC\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and wi

In [81]:
score_df = pd.DataFrame(scores)
score_df

,model,best score,best params
0,random_forest,0.968851,"{'max_depth': 100, 'n_estimators': 150}"
1,logistic_regression,0.926496,"{'C': 0.5, 'solver': 'newton-cg'}"
2,xgb,0.972080,"{'learning_rate': 0.3, 'n_estimators': 100}"
3,knn,0.862108,"{'algorithm': 'auto', 'n_neighbors': 15}"
